In [3]:
import arcade
import random

# Constants
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 400
BAR_WIDTH = 20
BAR_HEIGHT = 100
BAR_COLOR = arcade.color.BLUE
BALL_RADIUS = 20
BALL_COLOR = arcade.color.RED
BASKET_WIDTH = 60
BASKET_HEIGHT = 20
BASKET_COLOR = arcade.color.GREEN
MOVEMENT_SPEED = 5
FONT_SIZE = 20
VELOCITY = 10
class MyGame(arcade.Window):
    def __init__(self, width, height, title):
        super().__init__(width, height, title)
        self.bar_x = SCREEN_WIDTH - (SCREEN_WIDTH // 4)
        self.bar_y = SCREEN_HEIGHT - (SCREEN_HEIGHT // 4)
        self.background_color = arcade.color.LIGHT_GRAY
        self.bar_height = BAR_HEIGHT
        self.extend_bar = False
        self.shrink_bar = False
        self.move_left = False
        self.move_right = False
        self.score = 0
        self.is_grabbing = False

        # Randomize the initial position of the ball and basket and store them as attributes
        self.randomize_positions()

    def randomize_positions(self):
        # Randomize the ball's x-coordinate only in the left half of the screen
        self.ball_x = random.randint(BALL_RADIUS, SCREEN_WIDTH // 2 - BALL_RADIUS)
        
        # Randomize the basket's x-coordinate only in the right half of the screen
        self.basket_x = random.randint(SCREEN_WIDTH // 2 + BASKET_WIDTH // 2, SCREEN_WIDTH - BASKET_WIDTH // 2)
        
        # Ensure that the pillar's height is not taller than a portion of the screen
        max_pillar_height = SCREEN_HEIGHT // 2
        self.pillar_height = random.randint(50, max_pillar_height)
        
        # Set the initial y-coordinates for the ball and basket
        self.ball_y = BALL_RADIUS
        self.basket_y = BASKET_HEIGHT // 2

    def on_draw(self):
        arcade.start_render()
        # Draw the pillar in the middle of the screen with randomized height
        arcade.draw_rectangle_filled(SCREEN_WIDTH // 2, self.pillar_height // 2, 20, self.pillar_height, arcade.color.GRAY)
        arcade.draw_rectangle_filled(self.bar_x, self.bar_y, BAR_HEIGHT + 20, BAR_WIDTH, BAR_COLOR)
        arcade.draw_rectangle_filled(self.bar_x, self.bar_y + 40 - BAR_HEIGHT / 2 - self.bar_height / 2, BAR_WIDTH, self.bar_height, BAR_COLOR)
        arcade.draw_circle_filled(self.ball_x, self.ball_y, BALL_RADIUS, BALL_COLOR)
        arcade.draw_rectangle_filled(self.basket_x, self.basket_y, BASKET_WIDTH, BASKET_HEIGHT, BASKET_COLOR)
        arcade.draw_text(f"Bar Height: {self.bar_y - self.bar_height}", 10, SCREEN_HEIGHT - FONT_SIZE - 10, arcade.color.WHITE, FONT_SIZE)
        arcade.draw_text(f"Bar Height: {self.ball_y}", 10, SCREEN_HEIGHT - FONT_SIZE - 40, arcade.color.WHITE, FONT_SIZE)
        arcade.draw_text(f"Score: {self.score}", 10, SCREEN_HEIGHT - 2 * (FONT_SIZE + 25), arcade.color.WHITE, FONT_SIZE)
        
    def on_update(self, delta_time):
        if self.extend_bar:
            if self.bar_height <= self.bar_y:
                self.bar_height += 10
        if self.shrink_bar:
            if self.bar_height >= 10:
                self.bar_height -= 10
        if self.move_left:
            if (
                self.bar_x - MOVEMENT_SPEED - BAR_WIDTH / 2 > 0 and
                not self.check_collision_with_pillar(self.bar_x - MOVEMENT_SPEED, self.bar_y, self.bar_height)
            ):
                self.bar_x -= MOVEMENT_SPEED
        if self.move_right:
            if (
                self.bar_x + MOVEMENT_SPEED + BAR_WIDTH / 2 < SCREEN_WIDTH and
                not self.check_collision_with_pillar(self.bar_x + MOVEMENT_SPEED, self.bar_y, self.bar_height)
            ):
                self.bar_x += MOVEMENT_SPEED

        # Check if the growing bar is above the ball
        bar_above_ball = (
            self.bar_x - BAR_WIDTH < self.ball_x < self.bar_x + BAR_WIDTH and
            self.bar_y - self.bar_height <= self.ball_y + 20
        )

        if self.is_grabbing:
            if bar_above_ball:  # Only allow grabbing when the growing bar is above the ball
                self.ball_x = self.bar_x
                self.ball_y = self.bar_y - self.bar_height + BALL_RADIUS // 2 - 10
            else:
                self.is_grabbing = False  # Release the ball if the growing bar is not above the ball
        else:
            if self.ball_y >= BALL_RADIUS + 2:
                self.ball_y -= VELOCITY

        if (
            self.ball_x + BALL_RADIUS >= self.basket_x - BASKET_WIDTH / 2
            and self.ball_x - BALL_RADIUS <= self.basket_x + BASKET_WIDTH / 2
            and self.ball_y - BALL_RADIUS <= self.basket_y + BASKET_HEIGHT / 2
        ):
            self.score += 1
        
    def check_collision_with_pillar(self, x, y, bar_height):
        # Check if the bar would collide with the pillar at the given position
        return (
            x - BAR_WIDTH / 2 < SCREEN_WIDTH // 2 + 10 and
            x + BAR_WIDTH / 2 > SCREEN_WIDTH // 2 - 10 and
            y - bar_height < self.pillar_height
        )
    def on_key_press(self, key, modifiers):
        if key == arcade.key.DOWN:
            self.extend_bar = True
        elif key == arcade.key.UP:
            self.shrink_bar = True
        elif key == arcade.key.LEFT:
            self.move_left = True
        elif key == arcade.key.RIGHT:
            self.move_right = True
        elif key == arcade.key.SPACE:
            if not self.is_grabbing:
                self.is_grabbing = True
            else:
                self.is_grabbing = False
                
    def on_key_release(self, key, modifiers):
        if key == arcade.key.DOWN:
            self.extend_bar = False
        elif key == arcade.key.UP:
            self.shrink_bar = False
        elif key == arcade.key.LEFT:
            self.move_left = False
        elif key == arcade.key.RIGHT:
            self.move_right = False

def main():
    MyGame(SCREEN_WIDTH, SCREEN_HEIGHT, "BALL SORT CRANE GAME")
    arcade.run()

if __name__ == "__main__":
    main()